# QR Factorizations

In [1]:
using Random
using Statistics
using LinearAlgebra

In [2]:
# generate Householder u for a given x, normalizes it so u[1] = 1
function Householder(x)
    u = copy(x);
    σ = norm(x,2);
    u[1] += sign(x[1])*σ;
    u ./=u[1];
    β = 2/(u'u);
    return u, β, σ
end


Householder (generic function with 1 method)

In [3]:
function QR_Householder(A)
    # get dimensions
    m,n = size(A);
    R = copy(A); #avoid destroying A
    
    for i in 1:min(m-1,n)
        u, β, _ = Householder(R[i:end,i]);
        R[i:end, i:end] .= R[i:end,i:end] - β * u * (u' * R[i:end,i:end])
    end
    return triu(R[1:n, 1:n])
end

function QR_Householder_with_Q(A)
    # get dimensions
    m,n = size(A);
    R = copy(A); #avoid destroying A
    Qt = Matrix{Float64}(I, m, n)
    
    for i in 1:min(m-1,n)
        u, β, _ = Householder(R[i:end,i]);
        R[i:end, i:end] .-=  β * u * (u' * R[i:end,i:end])
        Qt[i:end, :] .-=  β* u * (u' *Qt[i:end, :])
    end
    return transpose(Qt), triu(R[1:n,1:n])
end


QR_Householder_with_Q (generic function with 1 method)

In [4]:
A = Float64[-1 1; 2 3]
display(A)
cond(A)

2×2 Matrix{Float64}:
 -1.0  1.0
  2.0  3.0

2.618033988749894

In [5]:
QR_Householder(A)

2×2 Matrix{Float64}:
 2.23607  2.23607
 0.0      2.23607

In [6]:
Q, R = QR_Householder_with_Q(A);
display(Q);
display(R);

2×2 transpose(::Matrix{Float64}) with eltype Float64:
 -0.447214  0.894427
  0.894427  0.447214

2×2 Matrix{Float64}:
 2.23607  2.23607
 0.0      2.23607

In [9]:
F = qr(A); 
display(F.Q);
display(F.R);

2×2 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.447214  0.894427
  0.894427  0.447214

2×2 Matrix{Float64}:
 2.23607  2.23607
 0.0      2.23607

In [14]:
Random.seed!(100)
A = randn(1000,1000);
cond(A)

1928.9310856062943

In [ ]:
Q, R = QR_Householder_with_Q(A);
F = qr(A);
@show norm(R -F.R);
@show norm(Q -F.Q);
@show norm(Q'Q-I); # Q̂ᵗQ̂-I, Q̂, the explicit numerical orthogonal matrix, measures numerical deficiency in orhtogonality
@show norm(Q*R-A); # Q̂R̂ - A = ΔA, the backwards error